### Data Hygiene
Checking if the outliers are Data Entry Errors. 

Sometimes a 500-hour delivery time is just a courier forgetting to hit "Complete" in the app until three weeks later.

In [13]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

from utils import read_orders

In [2]:
df = read_orders()

In [4]:
df['total_hours'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp'])
df['total_hours'] = df['total_hours'].dt.total_seconds() / 3600

df['rounded_time'] = df['order_delivered_customer_date'].dt.floor('min')
df['hour'] = df['rounded_time'].dt.hour 

In [5]:
df.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'total_hours', 'rounded_time', 'hour'],
      dtype='object')

In [6]:
cols = ['order_id', 'customer_id', 'order_delivered_customer_date', 'total_hours', 'rounded_time', 'hour']
df = df[cols]

In [7]:
df.head()

,order_id,customer_id,order_delivered_customer_date,total_hours,rounded_time,hour
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-10 21:25:13,202.477778,2017-10-10 21:25:00,21.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-08-07 15:27:45,330.768889,2018-08-07 15:27:00,15.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-17 18:06:29,225.461111,2018-08-17 18:06:00,18.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-12-02 00:28:42,317.010000,2017-12-02 00:28:00,0.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-16 18:17:02,68.973056,2018-02-16 18:17:00,18.0


#### Checking how much of the total orders is finished at night time

In [8]:
outside_work_hour = (df['hour'] >= 22) | (df['hour'] < 6)
df[outside_work_hour].shape

(16275, 6)

In [9]:
df.shape

(99441, 6)

In [10]:
16275 / 99441

0.16366488671674662

#### Consecutive Timestamp Analysis

In [11]:

# Sort by time
df = df.sort_values('rounded_time')

# Calculate difference in seconds between consecutive rows
df['time_diff'] = df['rounded_time'].diff().dt.total_seconds()

# If the time_diff is 0.0 multiple times in a row, it's a batch update
df['is_potential_error'] = df['time_diff'] == 0

In [12]:
pe = df[df['is_potential_error']]
print(pe.shape)
pe['order_delivered_customer_date'].value_counts()

(20827, 8)


order_delivered_customer_date
2018-05-14 20:02:44    3
2017-08-16 21:42:14    2
2018-06-20 19:22:15    2
2018-08-17 20:22:31    2
2017-06-19 18:47:51    2
                      ..
2017-12-20 16:14:40    1
2017-12-20 15:43:47    1
2017-12-20 15:42:05    1
2017-12-20 15:07:29    1
2018-08-31 02:32:20    1
Name: count, Length: 20654, dtype: int64

#### Finding
The count of orders that is completed within the same minute is extremely low, which is less than or equal 3.

And, the minutes are not at sharp hours. More like human input.

#### The "Post-Shift Completion" Theory

In [14]:
df.head()

,order_id,customer_id,order_delivered_customer_date,total_hours,rounded_time,hour,time_diff,is_potential_error
59102,36989eb07a0de2d3d3129eea35553875,aadd27185177fc7ac9b364898ac09343,2016-10-11 13:46:32,168.584167,2016-10-11 13:46:00,13.0,NaN,False
1384,7033745709b7cf1bac7d2533663592de,7f0ca17bb33b230b47459437cf0682c7,2016-10-11 14:46:49,168.557500,2016-10-11 14:46:00,14.0,3600.0,False
56143,d1eb8e4e276a4eea13a5c462c0765e60,9031f9dcde5860b34e6c65ac5c796d30,2016-10-13 03:10:34,199.668333,2016-10-13 03:10:00,3.0,131040.0,False
52382,92b44b87f1f7670b8911c5f0e642435e,e561a3f61440b031d3be286a696d06eb,2016-10-13 07:45:48,188.617222,2016-10-13 07:45:00,7.0,16500.0,False
91178,a9fc3ae13c740ad46532fd50b61c69d5,04a043776b45158df5f274435321c199,2016-10-13 15:44:57,209.023333,2016-10-13 15:44:00,15.0,28740.0,False


In [16]:
Q3 = df['total_hours'].quantile(0.75)
Q1 = df['total_hours'].quantile(0.25)
IQR = Q3 - Q1
df['is_late'] = df['total_hours'] > (Q3 + IQR * 1.5)

df['is_after_hours'] = (df['hour'] >= 22) | (df['hour'] < 6)
df.head()

,order_id,customer_id,order_delivered_customer_date,total_hours,rounded_time,hour,time_diff,is_potential_error,is_late,is_after_hours
59102,36989eb07a0de2d3d3129eea35553875,aadd27185177fc7ac9b364898ac09343,2016-10-11 13:46:32,168.584167,2016-10-11 13:46:00,13.0,NaN,False,False,False
1384,7033745709b7cf1bac7d2533663592de,7f0ca17bb33b230b47459437cf0682c7,2016-10-11 14:46:49,168.557500,2016-10-11 14:46:00,14.0,3600.0,False,False,False
56143,d1eb8e4e276a4eea13a5c462c0765e60,9031f9dcde5860b34e6c65ac5c796d30,2016-10-13 03:10:34,199.668333,2016-10-13 03:10:00,3.0,131040.0,False,False,True
52382,92b44b87f1f7670b8911c5f0e642435e,e561a3f61440b031d3be286a696d06eb,2016-10-13 07:45:48,188.617222,2016-10-13 07:45:00,7.0,16500.0,False,False,False
91178,a9fc3ae13c740ad46532fd50b61c69d5,04a043776b45158df5f274435321c199,2016-10-13 15:44:57,209.023333,2016-10-13 15:44:00,15.0,28740.0,False,False,False


In [17]:
# Create a contingency table
# Rows: On-Time, Late
# Cols: Work-Hours, After-Hours
contingency_table = pd.crosstab(df['is_late'], df['is_after_hours'])

chi2, p, dof, ex = chi2_contingency(contingency_table)
print(f"P-value: {p}")

P-value: 4.6431597105660776e-18


#### Finding

The Link is Real: There is a powerful, systematic relationship between an order being "Late" and it being recorded "After Hours."

Confirmation of Human Bias: This proves that the 16% of orders completed at night are not just a random mix of all orders. Instead, Late orders are disproportionately represented in that night-time group.

Root Cause Identified: You have moved from a "Delivery Delay" problem to a "Reporting Delay" problem. The drivers are likely finishing their physical work earlier but "closing" the difficult or delayed orders in the system only once they are off the road (The "Clean-Up Effect").

#### Action

Filter out all deliveries recorded after a certain hour (e.g., 9 PM) and re-run your delivery time distribution.

Compare the 'Cleaned' Mean vs. 'Original' Mean. You will likely find that the average delivery time drops significantly once you remove these human-entry outliers.